#### Import of libraries and uploading dataset from csv file

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
raw_data = pd.read_csv("/home/student/Pulpit/nypd.csv", low_memory=False)

In [50]:
col_list = ["NUMBER OF PERSONS KILLED","NUMBER OF PEDESTRIANS KILLED","NUMBER OF CYCLIST KILLED","NUMBER OF MOTORIST KILLED"]
df["Total killed"]=df[col_list].sum(axis=1)
col_list2 = ["NUMBER OF PERSONS INJURED","NUMBER OF PEDESTRIANS INJURED","NUMBER OF CYCLIST INJURED","NUMBER OF MOTORIST INJURED"]
df["Total injured"]=df[col_list2].sum(axis=1)
df

,ACCIDENT DATE,ACCIDENT TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,Total killed,Total injured
0,2019-08-05T00:00:00.000,16:30,QUEENS,11434,40.676052,-73.790184,"{'type': 'Point', 'coordinates': [-73.790184, ...",NaN,NaN,150-08 123 AVENUE,...,NaN,NaN,4184637,Sedan,Pick-up Truck,NaN,NaN,NaN,0.0,0.0
1,2019-08-27T00:00:00.000,16:02,BROOKLYN,11225,40.657780,-73.951096,"{'type': 'Point', 'coordinates': [-73.951096, ...",NaN,NaN,288 HAWTHORNE STREET,...,NaN,NaN,4195773,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,0.0,0.0
2,2019-08-15T00:00:00.000,17:57,MANHATTAN,10002,40.718143,-73.993835,"{'type': 'Point', 'coordinates': [-73.993835, ...",CHRYSTIE STREET,GRAND STREET,NaN,...,NaN,NaN,4202457,Sedan,NaN,NaN,NaN,NaN,0.0,2.0
3,2019-08-30T00:00:00.000,21:53,BRONX,10460,40.840534,-73.866610,"{'type': 'Point', 'coordinates': [-73.86661, 4...",NaN,NaN,1837 EAST TREMONT AVENUE,...,NaN,NaN,4198749,Taxi,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,0.0,0.0
4,2019-08-06T00:00:00.000,9:45,MANHATTAN,10016,40.745440,-73.975400,"{'type': 'Point', 'coordinates': [-73.9754, 40...",EAST 35 STREET,2 AVENUE,NaN,...,NaN,NaN,4183798,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612173,2012-07-21T00:00:00.000,22:46,BROOKLYN,11226,40.651741,-73.965241,"{'type': 'Point', 'coordinates': [-73.9652409,...",PARADE PLACE,CROOKE AVENUE,NaN,...,NaN,NaN,147595,SPORT UTILITY / STATION WAGON,UNKNOWN,NaN,NaN,NaN,0.0,0.0
1612174,2012-07-09T00:00:00.000,16:45,NaN,NaN,NaN,NaN,NaN,EDWARD L GRANT HIGHWAY,WEST 169 STREET,NaN,...,NaN,NaN,85220,PASSENGER VEHICLE,LIVERY VEHICLE,PASSENGER VEHICLE,NaN,NaN,0.0,10.0
1612175,2012-07-09T00:00:00.000,14:35,NaN,NaN,40.791492,-73.935763,"{'type': 'Point', 'coordinates': [-73.935763, ...",NaN,NaN,NaN,...,NaN,NaN,2853431,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN,0.0,0.0
1612176,2012-07-18T00:00:00.000,9:20,MANHATTAN,10024,40.779479,-73.973573,"{'type': 'Point', 'coordinates': [-73.9735729,...",CENTRAL PARK WEST,WEST 77 STREET,NaN,...,NaN,NaN,52405,SPORT UTILITY / STATION WAGON,VAN,BUS,NaN,NaN,0.0,0.0


In [75]:
data = df[["BOROUGH","CONTRIBUTING FACTOR VEHICLE 1", "Total killed", "Total injured"]]
data

,BOROUGH,CONTRIBUTING FACTOR VEHICLE 1,Total killed,Total injured
0,QUEENS,Passing Too Closely,0.0,0.0
1,BROOKLYN,Passing Too Closely,0.0,0.0
2,MANHATTAN,Driver Inattention/Distraction,0.0,2.0
3,BRONX,Unspecified,0.0,0.0
4,MANHATTAN,Driver Inattention/Distraction,0.0,2.0
...,...,...,...,...
1612173,BROOKLYN,Traffic Control Disregarded,0.0,0.0
1612174,NaN,Fatigued/Drowsy,0.0,10.0
1612175,NaN,Fatigued/Drowsy,0.0,0.0
1612176,MANHATTAN,Driver Inattention/Distraction,0.0,0.0


In [80]:
mask = data["Total killed"] <= 2.0
data = data[~mask]
mask2 = data["Total injured"] <= 2.0
data = data[~mask2]
data

,BOROUGH,CONTRIBUTING FACTOR VEHICLE 1,Total killed,Total injured
437105,BROOKLYN,Illnes,4.0,8.0
490565,MANHATTAN,Other Vehicular,16.0,24.0
536918,QUEENS,Unsafe Speed,8.0,30.0
619062,NaN,Unsafe Speed,4.0,16.0
651977,QUEENS,Traffic Control Disregarded,10.0,6.0
662436,NaN,Driver Inattention/Distraction,6.0,10.0
730319,NaN,Driver Inattention/Distraction,6.0,10.0
774965,NaN,Unsafe Speed,6.0,18.0
801409,NaN,Unsafe Speed,6.0,6.0
874084,NaN,Unsafe Speed,6.0,18.0


In [87]:
data_for_plot = data.dropna()
data_for_plot

,BOROUGH,CONTRIBUTING FACTOR VEHICLE 1,Total killed,Total injured
437105,BROOKLYN,Illnes,4.0,8.0
490565,MANHATTAN,Other Vehicular,16.0,24.0
536918,QUEENS,Unsafe Speed,8.0,30.0
651977,QUEENS,Traffic Control Disregarded,10.0,6.0
998199,QUEENS,Traffic Control Disregarded,6.0,6.0
1075852,QUEENS,Traffic Control Disregarded,6.0,6.0
1235204,BROOKLYN,Unspecified,4.0,4.0
1580295,STATEN ISLAND,Traffic Control Disregarded,4.0,4.0
1592515,STATEN ISLAND,Traffic Control Disregarded,4.0,4.0
